In [ ]:
# import lib

import os
import csv
import pandas as pd
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [ ]:
SCOPES = ["https://www.googleapis.com/auth/spreadsheets"]

In [ ]:
SPREADSHEET_ID = "1P8oONrbhDqPdj7ubLUWCmfXJd6eOexO5FzWWgOvzIlE"

In [ ]:
FORWARD_RANGE = 'Sheet1!Q3:T'  # Range for forward data (Q3 to T)
REVERSE_RANGE = 'Sheet1!W3:Z'  # Range for reverse data (W3 to Z)

In [ ]:
def main():
    creds = None
    # Load existing credentials from token.json or refresh them
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                "credentials.json", SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for future runs
        with open("token.json", "w") as token:
            token.write(creds.to_json())
    
    try:
        service = build("sheets", "v4", credentials=creds)
        sheet = service.spreadsheets()

        # Read data from the CSV file (two sheets: forward and reverse)
        csv_file = 'your-file.csv'  # Replace with your actual CSV file path
        forward_data = pd.read_csv(csv_file, sheet_name="forward")
        reverse_data = pd.read_csv(csv_file, sheet_name="reverse")

        # Convert the data to lists (as required by the Sheets API)
        forward_values = forward_data.values.tolist()
        reverse_values = reverse_data.values.tolist()

        # Clear the previous data in the forward range (Q3 to T)
        sheet.values().clear(spreadsheetId=SPREADSHEET_ID, range=FORWARD_RANGE).execute()

        # Update the forward sheet data (Q3 to T)
        sheet.values().update(
            spreadsheetId=SPREADSHEET_ID,
            range=FORWARD_RANGE,
            valueInputOption='RAW',
            body={'values': forward_values}
        ).execute()

        # Clear the previous data in the reverse range (W3 to Z)
        sheet.values().clear(spreadsheetId=SPREADSHEET_ID, range=REVERSE_RANGE).execute()

        # Update the reverse sheet data (W3 to Z)
        sheet.values().update(
            spreadsheetId=SPREADSHEET_ID,
            range=REVERSE_RANGE,
            valueInputOption='RAW',
            body={'values': reverse_values}
        ).execute()

        print("Data updated successfully.")

In [ ]:
if __name__ == "__main__":
    main()